In [31]:
from keras.datasets import imdb
from keras import preprocessing

In [32]:
max_features = 10000
max_len = 20

In [33]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

In [34]:
x_train = preprocessing.sequence.pad_sequences(x_train, max_len)
x_test = preprocessing.sequence.pad_sequences(x_test, max_len)

In [35]:
from keras.models import Sequential
from keras.layers import Embedding, Dense, Flatten

In [36]:
model = Sequential()
model.add(Embedding(max_features, 8, input_length=max_len))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [37]:
model.compile(
    optimizer='rmsprop',
    loss='binary_crossentropy',
    metrics=['acc']
)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 20, 8)             80000     
_________________________________________________________________
flatten_3 (Flatten)          (None, 160)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 161       
Total params: 80,161
Trainable params: 80,161
Non-trainable params: 0
_________________________________________________________________


In [38]:
history = model.fit(
    x_train, y_train,
    epochs = 10,
    batch_size=32,
    validation_split=0.2
)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 2s 115us/step - loss: 0.6759 - acc: 0.6050 - val_loss: 0.6398 - val_acc: 0.6814
Epoch 2/10
20000/20000 [==============================] - 2s 103us/step - loss: 0.5657 - acc: 0.7427 - val_loss: 0.5467 - val_acc: 0.7206
Epoch 3/10
20000/20000 [==============================] - 2s 103us/step - loss: 0.4752 - acc: 0.7808 - val_loss: 0.5113 - val_acc: 0.7384
Epoch 4/10
20000/20000 [==============================] - 2s 107us/step - loss: 0.4263 - acc: 0.8077 - val_loss: 0.5008 - val_acc: 0.7452
Epoch 5/10
20000/20000 [==============================] - 2s 104us/step - loss: 0.3930 - acc: 0.8258 - val_loss: 0.4981 - val_acc: 0.7538
Epoch 6/10
20000/20000 [==============================] - 2s 102us/step - loss: 0.3668 - acc: 0.8395 - val_loss: 0.5014 - val_acc: 0.7530
Epoch 7/10
20000/20000 [==============================] - 2s 104us/step - loss: 0.3435 - acc: 0.8533 - val_loss: 0.5052 - v

In [39]:
import os

In [40]:
imdb_dir = 'datasets/aclImdb'
train_dir = os.path.join(imdb_dir, 'train')

In [41]:
labels = []
texts = []

In [42]:
for label_type in ['neg', 'pos']:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname), encoding='utf8')
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)

In [43]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

In [44]:
max_len = 100
training_samples = 2000
validation_samples = 10000
max_words = 10000

In [45]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

In [46]:
word_index = tokenizer.word_index

In [47]:
data = pad_sequences(sequences, maxlen=max_len)
labels = np.asarray(labels)
print("Data Tensor shape : ", data.shape)
print("Label Tensor shape : ", labels.shape)

Data Tensor shape :  (25000, 100)
Label Tensor shape :  (25000,)


In [48]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

In [49]:
x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples: training_samples + validation_samples]
y_val = labels[training_samples: training_samples + validation_samples]

In [50]:
glove_dir = 'datasets/'

embedding_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.100D.txt'), encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embedding_index[word] = coefs
f.close()

embedding_dim = 100

In [51]:
print('%s개의 단어 벡터.' % len(embedding_index))

400000개의 단어 벡터.


In [52]:
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [53]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

In [54]:
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=max_len))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 100, 100)          1000000   
_________________________________________________________________
flatten_4 (Flatten)          (None, 10000)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 32)                320032    
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 33        
Total params: 1,320,065
Trainable params: 1,320,065
Non-trainable params: 0
_________________________________________________________________


In [55]:
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 100, 100)          1000000   
_________________________________________________________________
flatten_4 (Flatten)          (None, 10000)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 32)                320032    
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 33        
Total params: 1,320,065
Trainable params: 320,065
Non-trainable params: 1,000,000
_________________________________________________________________


In [56]:
model.compile(
    optimizer='rmsprop',
    loss='binary_crossentropy',
    metrics=['acc']
)
history = model.fit(
    x_train, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(x_val, y_val)
)
model.save_weights('pre_trained_glove_model.h5')

Train on 2000 samples, validate on 10000 samples
Epoch 1/10
2000/2000 [==============================] - 1s 439us/step - loss: 0.8117 - acc: 0.5565 - val_loss: 0.6869 - val_acc: 0.5325
Epoch 2/10
2000/2000 [==============================] - 1s 309us/step - loss: 0.6337 - acc: 0.6520 - val_loss: 0.6393 - val_acc: 0.6425
Epoch 3/10
2000/2000 [==============================] - 1s 345us/step - loss: 0.5327 - acc: 0.7445 - val_loss: 0.8831 - val_acc: 0.5405
Epoch 4/10
2000/2000 [==============================] - 1s 303us/step - loss: 0.4521 - acc: 0.7775 - val_loss: 0.6318 - val_acc: 0.6530
Epoch 5/10
2000/2000 [==============================] - 1s 372us/step - loss: 0.3708 - acc: 0.8490 - val_loss: 1.2047 - val_acc: 0.5281
Epoch 6/10
2000/2000 [==============================] - 1s 302us/step - loss: 0.3284 - acc: 0.8710 - val_loss: 0.8036 - val_acc: 0.6098
Epoch 7/10
2000/2000 [==============================] - 1s 386us/step - loss: 0.2725 - acc: 0.8905 - val_loss: 0.8113 - val_acc: 0.6270

In [58]:
test_dir = os.path.join(imdb_dir, 'test')
labels = []
texts = []

for label_type in ['neg', 'pos']:
    dir_name = os.path.join(test_dir, label_type)
    for fname in sorted(os.listdir(dir_name)):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname), encoding='utf8')
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)
                
sequences = tokenizer.texts_to_sequences(texts)
x_test = pad_sequences(sequences, maxlen=max_len)
y_test = np.asarray(labels)

In [60]:
model.load_weights('pre_trained_glove_model.h5')
model.evaluate(x_test, y_test)

25000/25000 [==============================] - 1s 49us/step


[1.171559064002037, 0.601]